[link text](https://)To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
# FastLanguageModel is an optimized way to load models.
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!

# this is for precision
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no  out-of-memory (OOMs) errors.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",  # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",  # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",  # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",  # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",  # Gemma 2x faster!
]  # More models at https://huggingface.co/unsloth


# load the model and tokanizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-27b",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf -  some models require an authentication token to access them.
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2025.2.15: Fast Gemma2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/199k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/7.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/7.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
# Adding LoRA Adapters - it just modifies the already-loaded model to prepare it for efficient fine-tuning. - gets the peft_model
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], #For best performance, keep all 7 layers as in the code.

    lora_alpha = 16, # 8 or 16 for a balance between stability and effectiveness - Higher values (e.g., 32) can lead to overfitting in small datasets.
    lora_dropout = 0, # Supports any, but = 0 (good for stable models and large datasets) is optimized - For small datasets, try lora_dropout = 0.05 or 0.1 to prevent overfitting. - If fine-tuning on large datasets, keeping it at 0 is fine.
    bias = "none",    # Supports any, but = "none" is optimized - Controls whether biases are trainable in LoRA layers - options: none, all, lora_only
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context - Saves GPU memory when training long sequences.
    random_state = 3407, # If reproducibility is important, keep it.
    use_rslora = False,  # We support rank stabilized LoRA - I think "Keep it False unless fine-tuning very large models (>30B parameters) & if LoRA model converges too slowly, try setting it to True"
    loftq_config = None, # And LoftQ - unless  fine-tune on low-memory hardware, keep it as None
)

Unsloth 2025.2.15 patched 46 layers with 46 QKV layers, 46 O layers and 46 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datasets import Dataset

# Define the EOS token
EOS_TOKEN = tokenizer.eos_token

# Define the prompt format
alpaca_prompt = """Translate the below text from Dialectal Arabic to Modern Standard Arabic:

### DA text:
{}

### MSA translation:
{}"""

# Load the dataset from Google Drive
csv_path = ""
df = pd.read_csv(csv_path, header=0)  # Ensure header=0 if the first row contains column names

# Define function to format the dataset
def formatting_prompts_func(examples):
    return {
        "text": [
            alpaca_prompt.format(da, msa) + EOS_TOKEN
            for da, msa in zip(examples["DA"], examples["MSA"])
        ]
    }

# Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Apply formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)

# Print first example to verify formatting
print(dataset[0])



Map:   0%|          | 0/62775 [00:00<?, ? examples/s]

{'Dialect': 'EGY', 'DA': 'عاجبني البلوفر بتاعك.', 'MSA': 'أحب الجاكت الخاص بك .', 'text': 'Translate the below text from Dialectal Arabic to Modern Standard Arabic:\n\n### DA text:\nعاجبني البلوفر بتاعك.\n\n### MSA translation:\nأحب الجاكت الخاص بك .<eos>'}


In [ ]:
print(dataset["text"][0])

Translate the below text from Dialectal Arabic to Modern Standard Arabic:

### DA text:
عاجبني البلوفر بتاعك.

### MSA translation:
أحب الجاكت الخاص بك .<eos>


In [ ]:
dataset

Dataset({
    features: ['Dialect', 'DA', 'MSA', 'text'],
    num_rows: 62775
})

In [ ]:
dataset['text']

['Translate the below text from Dialectal Arabic to Modern Standard Arabic:\n\n### DA text:\nعاجبني البلوفر بتاعك.\n\n### MSA translation:\nأحب الجاكت الخاص بك .<eos>',
 'Translate the below text from Dialectal Arabic to Modern Standard Arabic:\n\n### DA text:\nلا مستحيل احكي هيك\n\n### MSA translation:\nلا من المستحيل أن أتكلم هكذا<eos>',
 'Translate the below text from Dialectal Arabic to Modern Standard Arabic:\n\n### DA text:\nعلى الشمال، لو سمحت.\n\n### MSA translation:\nإلى اليسار ، من فضلك .<eos>',
 'Translate the below text from Dialectal Arabic to Modern Standard Arabic:\n\n### DA text:\nفيك تحط تخت زيادة بالأوضة؟\n\n### MSA translation:\nهل من الممكن أن تضع سريرا إضافيا في الغرفة ؟<eos>',
 'Translate the below text from Dialectal Arabic to Modern Standard Arabic:\n\n### DA text:\nكتير هيك.\n\n### MSA translation:\nإنه كثير جداً .<eos>',
 'Translate the below text from Dialectal Arabic to Modern Standard Arabic:\n\n### DA text:\nعندك واين من كاليفورنيا؟\n\n### MSA translation:

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 2, # Simulates a batch size of 2 x 4 = 8 by accumulating gradients before updating weights.
        warmup_steps = 5,
        num_train_epochs=1,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(), # uses bf16 (a floating-point formats) for effiency if supported
        logging_steps = 1, # number of training steps that should pass before the training process logs information like loss, metrics, and other relevant data to the console
        optim = "adamw_8bit", # An 8-bit version of AdamW to reduce memory usage.
        weight_decay = 0.01, # Think if I need to change this - Helps prevent overfitting.
        lr_scheduler_type = "linear", # Reduces learning rate gradually.
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/62775 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/62775 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/62775 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/62775 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
15.209 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 62,775 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 2
\        /    Total batch size = 32 | Total steps = 1,962
 "-____-"     Number of trainable parameters = 114,180,096
AUTOTUNE bmm(512x89x128, 512x128x89)
  triton_bmm_14 0.0686 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=128, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_3 0.0727 ms 94.4% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=32, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=8
  triton_bmm_10 0.0737 ms 93.1% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=128, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_5 0.0758 ms 90.5% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=16, BLOCK_

Step,Training Loss
1,3.185500
2,3.313100
3,3.191000
4,3.221200
5,3.144300
6,2.997000
7,2.729100
8,2.575600
9,2.476100
10,2.338900


AUTOTUNE bmm(512x128x128, 512x128x128)
  bmm 0.0594 ms 100.0% 
  triton_bmm_166 0.0707 ms 84.1% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=128, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_162 0.0788 ms 75.3% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=128, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_165 0.0799 ms 74.4% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=128, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=3, num_warps=4
  triton_bmm_157 0.0809 ms 73.4% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_167 0.0860 ms 69.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=128, BLOCK_N=128, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

6852.0208 seconds used for training.
114.2 minutes used for training.
Peak reserved memory = 18.08 GB.
Peak reserved memory for training = 2.871 GB.
Peak reserved memory % of max memory = 45.706 %.
Peak reserved memory for training % of max memory = 7.258 %.


<a name="Inference"></a>
### Inference Test
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "بدي هدول الأواعي منضفين اليوم المسا.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda") # it Converts text input into tokenized tensors and moves them to GPU (cuda).

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
example_output = tokenizer.batch_decode(outputs)
example_output

AUTOTUNE bmm(32x41x128, 32x128x41)
  triton_bmm_457 0.0113 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=128, BLOCK_M=32, BLOCK_N=32, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_458 0.0113 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=32, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=8
  triton_bmm_459 0.0113 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=32, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=8
  triton_bmm_463 0.0113 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=64, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=3, num_warps=8
  triton_bmm_462 0.0123 ms 91.7% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_465 0.0

['<bos>Translate the below text from Dialectal Arabic to Modern Standard Arabic:\n\n### DA text:\nبدي هدول الأواعي منضفين اليوم المسا.\n\n### MSA translation:\nأريد هذه الملابس أن تغسل لي هذا المساء .<eos>']

<a name="Run on Test Set"></a>
### Run on Test Set
Run the model on Test Set and save predicted transaltions to google drive.
The transaltions are evaluated using a seperate notwbook.

In [ ]:
from tqdm import tqdm  # Import tqdm for progress bar

# Load tokenizer and model
FastLanguageModel.for_inference(model)  # Enable faster inference

# Load the test dataset
test_file_path = ""
test_data = pd.read_csv(test_file_path)

# Ensure 'DA' column exists
assert "DA" in test_data.columns, "Missing 'DA' column in the dataset!"

# Prepare the prompt template
alpaca_prompt = """Translate the below text from Dialectal Arabic to Modern Standard Arabic:

### DA text:
{}

### MSA:
"""

# Function for inference with progress bar
def generate_translation(da_text):
    input_text = alpaca_prompt.format(da_text)
    inputs = tokenizer([input_text], return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)

    predicted_msa = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract only the generated MSA text
    return predicted_msa.split("### MSA:")[-1].strip()

# Apply inference with tqdm progress bar
tqdm.pandas(desc="Generating Translations")
test_data["Predicted MSA"] = test_data["DA"].progress_apply(generate_translation)

# Save the results
output_file_path = ""
test_data.to_csv(output_file_path, index=False)

print(f"Predicted translations saved to: {output_file_path}")


Generating Translations:   2%|▏         | 27/1200 [00:41<25:42,  1.32s/it]W0302 19:19:21.161000 271 torch/utils/_sympy/interp.py:159] [0/7] failed while executing pow_by_natural([VR[4, int_oo], VR[-1, -1]])
W0302 19:19:21.175000 271 torch/utils/_sympy/interp.py:159] [0/7] failed while executing pow_by_natural([VR[4, int_oo], VR[-1, -1]])
AUTOTUNE bmm(32x32x128, 32x128x32)
  triton_bmm_491 0.0092 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=128, BLOCK_M=32, BLOCK_N=32, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=4
  triton_bmm_497 0.0092 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=128, BLOCK_M=32, BLOCK_N=32, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=4
  triton_bmm_489 0.0102 ms 90.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=128, BLOCK_M=32, BLOCK_N=32, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_495 0.0102 ms 90.0% ACC_TYPE='tl.float32', 

Predicted translations saved to: /content/drive/My Drive/Grad Project/Fine Tuning Predicted Translations/EXP5/exp5-gemma2-27b-test-output.csv


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

model.push_to_hub("", token = "") # Online saving
tokenizer.push_to_hub("", token = "") # Online saving

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

Saved model to https://huggingface.co/er-abd/lora_model_gemma2_27b_extended_gold_dataset


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
    [
        alpaca_prompt.format(
            "What is a famous tall tower in Paris?",  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous tall tower in Paris, France. It is located on the Champ de Mars in the 7th arrondissement of Paris, on the left bank of the River Seine. The tower was designed by engineer Gustave Eiffel and his company, and was constructed from 1887 to 18']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# This merges the LoRA adapters into the base model - Saves the full model (instead of just LoRA adapters) in 16-bit precision.
# The model is saved in the "model/" directory
# 16-bit merging reduces storage & memory usage compared to 32-bit

# save_pretrained_merged: Saves a 16-bit merged model locally.	- alloes to Load model without LoRA.
# push_to_hub_merged: Uploads the merged 16-bit model to Hugging Face.	- Share & deploy easily.


# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
